# Initial Import

In [1]:
from tqdm import tqdm
import os
import numpy as np
import torch
from torch import optim
from torch.utils.data import Dataset, DataLoader, TensorDataset, random_split
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd

from main import backgammon

/Applications/anaconda3/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


pygame 2.5.2 (SDL 2.28.3, Python 3.9.12)
Hello from the pygame community. https://www.pygame.org/contribute.html


## Import Dataset

In [2]:
def load_dataset(file_path):
    # Load CSV using pandas
    df = pd.read_csv(file_path)
    
    # Split into features and labels
    data = df.values
    X = torch.tensor(data[:, :-1], dtype=torch.float32)
    y = torch.tensor(data[:, -1], dtype=torch.float32).unsqueeze(1)  # Ensure shape (N, 1)
    
    return TensorDataset(X, y)

# train_file = os.path.join("..","Data","Deep","BearoffRace",'train.txt')
# validation_file = os.path.join("..","Data","Deep","BearoffRace",'validation.txt')
# test_file = os.path.join("..","Data","Deep","BearoffRace",'test.txt')

# train_dataset = load_dataset(train_file)
# validation_dataset = load_dataset(validation_file)
# test_dataset = load_dataset(test_file)

# train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
# val_loader = DataLoader(validation_dataset, batch_size=64)
# test_loader = DataLoader(test_dataset, batch_size=64)

# # Example: iterate over a batch from the train set
# for X_batch, y_batch in train_loader:
#     print("Input shape:", X_batch.shape)
#     print("Labels shape:", y_batch.shape)
#     break

In [3]:
class BGNet(nn.Module):
    def __init__(self):
        super(BGNet, self).__init__()
        # Shared input-to-hidden layer: both board states use the same weights.
        self.fc_shared = nn.Linear(289, 12)
        # Hidden-to-output layer: we use a single layer that will be applied
        # to both hidden representations. In the forward pass we explicitly
        # flip the sign for one branch to enforce that its effect is the negative.
        self.fc_out = nn.Linear(12, 1)

    def forward(self, board_left, board_right):
        # Process left board: apply shared fc layer and ReLU activation.
        h_left = F.relu(self.fc_shared(board_left))
        # Process right board: same shared fc layer.
        h_right = F.relu(self.fc_shared(board_right))
        
        # Compute the output for each branch using the same fc_out layer.
        out_left = self.fc_out(h_left)
        # Explicitly invert the output from the right branch.
        out_right = -self.fc_out(h_right)
        
        # The network's decision is the sum of these contributions.
        # This is equivalent to computing the difference between the two evaluations.
        diff = out_left + out_right
        
        # Apply a sigmoid activation so that final output is between 0 and 1.
        final_output = torch.sigmoid(diff)
        return final_output

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim

def evaluate(model, data_loader, device, optimizer, criterion):
    model.eval()
    total_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for X_batch, y_batch in data_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            outputs = model(X_batch[:, :289], X_batch[:, 289:])
            loss = criterion(outputs, y_batch)
            total_loss += loss.item() * y_batch.size(0)

            preds = (outputs > 0.5).float()
            correct += (preds == y_batch).sum().item()
            total += y_batch.size(0)

    avg_loss = total_loss / total
    accuracy = correct / total
    return avg_loss, accuracy

def train(num_epochs, train_loader, model, criterion, optimizer, val_loader, test_loader, model_type):
    # === Training Loop ===
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    for epoch in range(1, num_epochs + 1):
        model.train()
        running_loss = 0
        correct = 0
        total = 0

        for X_batch, y_batch in train_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            
            board_left = X_batch[:, :289]
            board_right = X_batch[:, 289:]

            outputs = model(board_left, board_right)
            loss = criterion(outputs, y_batch)
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item() * y_batch.size(0)

            # For training accuracy
            preds = (outputs > 0.5).float()
            correct += (preds == y_batch).sum().item()
            total += y_batch.size(0)

        avg_train_loss = running_loss / total
        train_acc = correct / total

        if epoch % 10 == 0:
            val_loss, val_acc = evaluate(model, val_loader, device, optimizer, criterion)
            test_loss, test_acc = evaluate(model, test_loader, device, optimizer, criterion)
            
            print(f"Epoch {epoch:03d}:")
            print(f"  Train Loss: {avg_train_loss:.4f}, Accuracy: {train_acc*100:.2f}%")
            print(f"  Val   Loss: {val_loss:.4f}, Accuracy: {val_acc*100:.2f}%")
            print(f"  Test  Loss: {test_loss:.4f}, Accuracy: {test_acc*100:.2f}%\n")

        torch.save(model.state_dict(), f"{model_type}.pth")


In [ ]:
# Contact Model

train_file = os.path.join("..","Data","Deep","Contact",'train.txt')
validation_file = os.path.join("..","Data","Deep","Contact",'validation.txt')
test_file = os.path.join("..","Data","Deep","Contact",'test.txt')

train_dataset = load_dataset(train_file)
validation_dataset = load_dataset(validation_file)
test_dataset = load_dataset(test_file)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(validation_dataset, batch_size=64)
test_loader = DataLoader(test_dataset, batch_size=64)
contact_model = BGNet()

# Loss function: Binary Cross-Entropy (good for sigmoid output)
criterion = nn.BCELoss()

# Optimizer
optimizer = optim.Adam(contact_model.parameters(), lr=0.001)

# Training settings
num_epochs = 50  # Change as needed
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
contact_model.to(device)
train(num_epochs, train_loader, contact_model, criterion, optimizer, val_loader, test_loader, model_type="contact_50")

Epoch 010:
  Train Loss: 0.4884, Accuracy: 76.70%
  Val   Loss: 0.4744, Accuracy: 77.92%
  Test  Loss: 0.6457, Accuracy: 67.21%

Epoch 020:
  Train Loss: 0.4208, Accuracy: 80.92%
  Val   Loss: 0.4090, Accuracy: 81.70%
  Test  Loss: 0.7100, Accuracy: 65.93%

Epoch 030:
  Train Loss: 0.3846, Accuracy: 82.98%
  Val   Loss: 0.3739, Accuracy: 83.50%
  Test  Loss: 0.7792, Accuracy: 65.46%

Epoch 040:
  Train Loss: 0.3592, Accuracy: 84.80%
  Val   Loss: 0.3483, Accuracy: 85.28%
  Test  Loss: 0.8283, Accuracy: 64.18%

Epoch 050:
  Train Loss: 0.3381, Accuracy: 85.64%
  Val   Loss: 0.3261, Accuracy: 86.59%
  Test  Loss: 0.8745, Accuracy: 64.99%

Epoch 060:
  Train Loss: 0.3212, Accuracy: 86.82%
  Val   Loss: 0.3110, Accuracy: 87.36%
  Test  Loss: 0.9288, Accuracy: 64.18%

Epoch 070:
  Train Loss: 0.3051, Accuracy: 87.52%
  Val   Loss: 0.2932, Accuracy: 88.12%
  Test  Loss: 0.9659, Accuracy: 63.13%

Epoch 080:
  Train Loss: 0.2904, Accuracy: 88.15%
  Val   Loss: 0.2784, Accuracy: 89.07%
  Test  

In [ ]:
# Midboard Race Model

train_file = os.path.join("..","Data","Deep","MidboardRace",'train.txt')
validation_file = os.path.join("..","Data","Deep","MidboardRace",'validation.txt')
test_file = os.path.join("..","Data","Deep","MidboardRace",'test.txt')

train_dataset = load_dataset(train_file)
validation_dataset = load_dataset(validation_file)
test_dataset = load_dataset(test_file)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(validation_dataset, batch_size=64)
test_loader = DataLoader(test_dataset, batch_size=64)
midboard_race_model = BGNet()

# Loss function: Binary Cross-Entropy (good for sigmoid output)
criterion = nn.BCELoss()

# Optimizer
optimizer = optim.Adam(midboard_race_model.parameters(), lr=0.001)

# Training settings
num_epochs = 50  # Change as needed
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
midboard_race_model.to(device)
train(num_epochs, train_loader, midboard_race_model, criterion, optimizer, val_loader, test_loader, model_type="midboard_race_50")

Epoch 010:
  Train Loss: 0.4208, Accuracy: 81.77%
  Val   Loss: 0.4061, Accuracy: 82.62%
  Test  Loss: 0.4386, Accuracy: 79.65%

Epoch 020:
  Train Loss: 0.3955, Accuracy: 83.12%
  Val   Loss: 0.3828, Accuracy: 83.76%
  Test  Loss: 0.4174, Accuracy: 81.16%

Epoch 030:
  Train Loss: 0.3879, Accuracy: 83.78%
  Val   Loss: 0.3744, Accuracy: 84.63%
  Test  Loss: 0.4089, Accuracy: 81.41%

Epoch 040:
  Train Loss: 0.3827, Accuracy: 84.07%
  Val   Loss: 0.3693, Accuracy: 84.37%
  Test  Loss: 0.4075, Accuracy: 82.16%

Epoch 050:
  Train Loss: 0.3789, Accuracy: 84.33%
  Val   Loss: 0.3651, Accuracy: 85.07%
  Test  Loss: 0.4041, Accuracy: 82.16%

Epoch 060:
  Train Loss: 0.3755, Accuracy: 84.71%
  Val   Loss: 0.3603, Accuracy: 85.07%
  Test  Loss: 0.4067, Accuracy: 82.66%

Epoch 070:
  Train Loss: 0.3708, Accuracy: 85.05%
  Val   Loss: 0.3554, Accuracy: 85.85%
  Test  Loss: 0.4031, Accuracy: 83.42%

Epoch 080:
  Train Loss: 0.3657, Accuracy: 85.26%
  Val   Loss: 0.3481, Accuracy: 86.72%
  Test  

In [ ]:
# Bearoff
train_file = os.path.join("..","Data","Deep","BearoffRace",'train.txt')
validation_file = os.path.join("..","Data","Deep","BearoffRace",'validation.txt')
test_file = os.path.join("..","Data","Deep","BearoffRace",'test.txt')

train_dataset = load_dataset(train_file)
validation_dataset = load_dataset(validation_file)
test_dataset = load_dataset(test_file)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(validation_dataset, batch_size=64)
test_loader = DataLoader(test_dataset, batch_size=64)

bearoff_race_model = BGNet()

# Loss function: Binary Cross-Entropy (good for sigmoid output)
criterion = nn.BCELoss()

# Optimizer
optimizer = optim.Adam(bearoff_race_model.parameters(), lr=0.001)

# Training settings
num_epochs = 50  # Change as needed
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
bearoff_race_model.to(device)
train(num_epochs, train_loader, bearoff_race_model, criterion, optimizer, val_loader, test_loader, model_type="bearoff_race_50")

Epoch 010:
  Train Loss: 0.2698, Accuracy: 89.66%
  Val   Loss: 0.2621, Accuracy: 89.77%
  Test  Loss: 0.2957, Accuracy: 86.62%

Epoch 020:
  Train Loss: 0.1857, Accuracy: 92.67%
  Val   Loss: 0.1804, Accuracy: 92.92%
  Test  Loss: 0.2101, Accuracy: 91.52%

Epoch 030:
  Train Loss: 0.1384, Accuracy: 94.79%
  Val   Loss: 0.1341, Accuracy: 95.04%
  Test  Loss: 0.1570, Accuracy: 93.15%

Epoch 040:
  Train Loss: 0.1153, Accuracy: 95.53%
  Val   Loss: 0.1117, Accuracy: 96.00%
  Test  Loss: 0.1416, Accuracy: 94.13%

Epoch 050:
  Train Loss: 0.1030, Accuracy: 96.26%
  Val   Loss: 0.1003, Accuracy: 96.33%
  Test  Loss: 0.1318, Accuracy: 94.29%

Epoch 060:
  Train Loss: 0.0948, Accuracy: 96.68%
  Val   Loss: 0.0929, Accuracy: 96.69%
  Test  Loss: 0.1340, Accuracy: 93.64%

Epoch 070:
  Train Loss: 0.0901, Accuracy: 96.88%
  Val   Loss: 0.0875, Accuracy: 96.90%
  Test  Loss: 0.1360, Accuracy: 94.29%

Epoch 080:
  Train Loss: 0.0866, Accuracy: 96.86%
  Val   Loss: 0.0827, Accuracy: 96.99%
  Test  